# iCAT Workflow
---
A walkthrough for getting started with the iCAT post-processing workflow on the sonic server. This walkthrough assumes that both `render-ws` and CATMAID are up and running successfully and that you have gone through the [`iCAT-startup`](https://github.com/lanery/iCAT-workflow/blob/master/docs/iCAT-startup.md) guide (or that someone else has on your behalf).

First, check that both `render-ws` and CATMAID are running.
```
systemctl status render
systemctl status catmaid
```
And if you get the green light, go to the homepages for `render-ws` and CATMAID

| Service     | Homepage                                      |
| ----------: | --------------------------------------------- |
| `render-ws` | http://sonic:8080/render-ws/view/index.html   |
| CATMAID     | http://sonic/catmaid/                         |

Also make sure that the `icat` conda environment is the active environment.

---
## Clone this repository
This repository contains sample data and scripts helpful for running through an introductory iCAT workflow/project. Assuming you are somewhere in your home directory
```
git clone https://github.com/lanery/iCAT-workflow.git
```

#### Sample Data
The sample data is organized such that each "stack" of images is stored in its own directory. A "stack" is a key concept within the `render-ws` + CATMAID ecosystem that will be expanded upon later. But basically, a stack can be thought of a collection of **one or more** *images* at **one or more** *layers in z* from **one** *imaging channel*. Hence, all of the small EM tiles will belong to one stack as will the large EM tiles, as will each fluorescence channel. It is perhaps easiest to grasp this organization scheme by looking at the directory tree of the sample data (`iCAT-workflow/iCAT_sample_data`)
```
iCAT-workflow
└───┬ iCAT_sample_data
    ├───┬ amylase
    │   └─── amylase-00000x00000.ome.tif
    ├───┬ big_EM
    │   └──── big_EM-00000x00000.ome.tif
    ├───┬ hoechst
    │   └──── hoechst-00000x00000.ome.tif
    ├───┬ insulin
    │   └──── insulin-00000x00000.ome.tif
    └───┬ lil_EM
        ├──── lil_EM-00008x00011.ome.tif
        ├──── lil_EM-00008x00012.ome.tif
        ├──── ...
        ├──── lil_EM-00012x00015.ome.tif
        └──── lil_EM-00012x00016.ome.tif

```
This is (at least for now) the optimal organization scheme for working with image data in the workflow. Unfortunately, it is not how raw data is output by Odemis. How to go from raw Odemis data to nicely organized, iCAT-friendly data will be covered later.

It is assumed that if you are going to be viewing your data in CATMAID, then it is worthy of long term storage. Even though this is just sample data, we will treat it as if it were a real project. Thus, copy the sample data to your long term storage folder.
```
cd ./iCAT-workflow/
cp -a ./iCAT_sample_data/ /long_term_storage/<user>/<data storage folder>/
```

---
## First render-python project
A project can have multiple stacks and each stack can contain multiple z layers with each layer containing multiple image tiles. The organizational structure of each stack, its layers, and its layers' tiles are all stored as metadata in a database accessed via http requests. Or something like that. We will now go through an interactive `render-python` session to get a feeling for what working with `render-ws` is like. Note that `render-python` is essentially just a regular python package (one that happens to make a lot of calls to a java library) and so it is not inherently interactive. You could instead run `render-python` with scripts, a package of your own, or make use of the Allen Institute's [`render-python-apps`](https://github.com/AllenInstitute/render-python-apps/tree/master/renderapps) repository.

### 0 Imports
Import the `render-python` api as well as some other useful libraries

In [1]:
# Libraries needed
import re
from pathlib import Path
import renderapi
from renderapi.layout import Layout
from renderapi.transform import AffineModel
from renderapi.tilespec import TileSpec

### 1 Stack Data
Practically speaking, this is not really how you would store data about your stack. `<Future documentation>` goes over good practices for both generating the necessary stack data as well as importing stacks to `render`. But since this is the first time, we'll keep things simple and pretend it is this easy in practice. Anyway, skim through `stack_data` to get a feel for what information a stack contains. But note that we infer additional information about a stack from these basic parameters, and that the data below is just that which is difficult to infer without parsing the metadata.

In the second line, change `<user>` to your username.

In [3]:
stack_data = {
    'data_dir': Path('/long_term_storage/<user>/SECOM/iCAT_sample_data'),  # <--- substitute your username here
    'tile_convention': '{stack}/{stack}-{c}x{r}.ome.tif',
    'width': 2048,
    'height': 2048,
    'N_sections': 1,
    'lil_EM': {
        'px_size': 4.829,  # nm/px
        'overlap': 20,  # %
        'intensity_range': (31800, 35200),
        'scopeId': 'Verios',
        'cameraId': 'TLD',
    },
    'big_EM': {
        'px_size': 84.9,  # nm/px
        'overlap': 20,  # %
        'intensity_range': (30400, 32100),
        'scopeId': 'Verios',
        'cameraId': 'TLD',
    },
    'hoechst': {
        'px_size': 99.6,  # nm/px
        'overlap': 20,  # %
        'intensity_range': (2500, 15400),
        'scopeId': 'SECOM',
        'cameraId': 'Andor',
    },
    'amylase': {
        'px_size': 99.6,  # nm/px
        'overlap': 20,  # %
        'intensity_range': (2000, 8000),
        'scopeId': 'SECOM',
        'cameraId': 'Andor',
    },
    'insulin': {
        'px_size': 99.6,  # nm/px
        'overlap': 20,  # %
        'intensity_range': (1200, 5000),
        'scopeId': 'SECOM',
        'cameraId': 'Andor',
    }
}

### 2 Create (Empty) render Stacks
Next we will create empty stacks within `render`. To do so, we first have to supply `render-python` with some configuration info. All you have to do is change `<user>` in the line `'owner': <user>` to your username. If you don't, you will get an error about how the `<` and `>` characters are not allowed.

In [7]:
# Create a renderapi.connect.Render object
render_connect_params = {
    'host': 'sonic',
    'port': 8080,
    'owner': '<user>',  # <--- Substitute your username here
    'project': 'iCAT_demo',
    'client_scripts': \
        '/home/catmaid/render/render-ws-java-client/src/main/scripts',
    'memGB': '2G'
}
render = renderapi.connect(**render_connect_params)

# Create (empty) stacks
stacks = ['lil_EM',
          'big_EM',
          'hoechst',
          'amylase',
          'insulin']

for stack in stacks:
    renderapi.stack.create_stack(stack, render=render)

Now go to the `render-ws` [homepage](http://sonic:8080/render-ws/view/index.html) and click on `Render Project Dashboard`. This will open a new tab and you should see a table with the names of the stacks in the script all in the `LOADING` state. You can view information about each stack by clicking on `View --> Metadata` (or by going to `http://sonic:8080/render-ws/v1/owner/<user>/project/iCAT_demo/stack/<stack>`).

### 3 Generate Tile Specifications
Here we define a helper function to help generate a list of what are called `TileSpecs`. A `TileSpec` (short for tile specification) is a `render-python` object with many of the same properties/parameters written out in the `stack_data`, but each individual image tile has its own specifications.

In [13]:
def gen_tile_specs(stack, stack_data):
    """
    Helper function for generating TileSpecs
    
    Reads in stack data and then loops through all the image tiles in a
    stack directory to populate the list of TileSpecs
    """
    # Get input from stack_data
    data_dir = stack_data['data_dir']
    N_sections = stack_data.get('N_sections', 1)
    width = stack_data['width']
    height = stack_data['height']
    overlap = stack_data.get('overlap', 20)
    px_size = stack_data[stack]['px_size']
    intensity_range = stack_data[stack].get('intensity_range', (0, 65535))

    # TODO: use tile_convention instead
    tiles = list(data_dir.glob(f'{stack}/*.ome.tif'))
    tile_specs = []
    for z in range(N_sections):
        for tile in tiles:

            # TODO: use tile_convention instead
            c, r = [int(i) for i in re.findall('\d+', tile.name)]
            x_pos = c * width * (1 - overlap) * px_size
            y_pos = r * height * (1 - overlap) * px_size

            layout = Layout(sectionId=f'{z:05d}',
                            scopeId=stack_data.get('scopeId'),
                            cameraId=stack_data.get('cameraId'),
                            imageRow=r,
                            imageCol=c,
                            stageX=x_pos,
                            stageY=y_pos,
                            rotation=0.0,
                            pixelsize=px_size)

            at = AffineModel(B0=layout.stageX/layout.pixelsize,
                             B1=layout.stageY/layout.pixelsize)

            tileId = tile.name.split('.')[0]
            imageUrl = tile.as_uri()
            tile_spec = TileSpec(tileId=tileId,
                                 z=z,
                                 width=width,
                                 height=height,
                                 minint=intensity_range[0],
                                 maxint=intensity_range[1],
                                 imageUrl=imageUrl,
                                 maskUrl=None,
                                 layout=layout,
                                 tforms=[at])

            tile_specs.append(tile_spec)

    return tile_specs

Now we use the `gen_tile_specs` function we just defined to generate the tile specifications.

In [15]:
# Use gen_tile_specs to generate TileSpecs
tile_specs = {}
for stack in stacks:

    # Get TileSpecs from stack_data
    tile_specs[stack] = gen_tile_specs(stack, stack_data)

Now that we have generated all of the tile specifications, lets take a look at what a tile specification looks like. You can replace `'big_EM'` with `'hoechst'`, `'insulin'`, `'amylase'`, or `'lil_EM'` to see what a `TileSpec` in each of these respective stacks looks like.

In [16]:
tile_specs['big_EM'][0].to_dict()

{'tileId': 'big_EM-00000x00000',
 'z': 0,
 'width': 2048,
 'height': 2048,
 'minIntensity': 30400,
 'maxIntensity': 32100,
 'layout': {'sectionId': '00000',
  'imageRow': 0,
  'imageCol': 0,
  'stageX': 0.0,
  'stageY': 0.0,
  'rotation': 0.0,
  'pixelsize': 84.9},
 'mipmapLevels': {'0': {'imageUrl': 'file:///long_term_storage/rlane/SECOM/iCAT_sample_data/big_EM/big_EM-00000x00000.ome.tif'}},
 'transforms': {'type': 'list',
  'specList': [{'type': 'leaf',
    'className': 'mpicbg.trakem2.transform.AffineModel2D',
    'dataString': '1.0000000000 0.0000000000 0.0000000000 1.0000000000 0.0000000000 0.0000000000'}]}}

### 4 Import Tile Specifications to render
Now we have created the `render-python` `TileSpec` objects, but they still have to be imported into `render-ws`. Thankfully, `render-python` makes this very easy and takes care of it behind the scenes. We will also change the stack state from `LOADING` to `COMPLETE` to tell `render-ws` we are done modifying these stacks.

In [17]:
for stack in stacks:

    # Import TileSpecs to render
    renderapi.client.import_tilespecs(stack,
                                      tile_specs[stack],
                                      close_stack=True,
                                      render=render)

    # Set stack state to complete
    renderapi.stack.set_stack_state(stack, 'COMPLETE', render=render)

### 5 View Stacks in render
Now we can see the same TileSpec information via a `render-ws` http request:  
`http://sonic:8080/render-ws/v1/owner/<user>/project/iCAT_demo/stack/big_EM/tile/big_EM-00000x00000`  
(again replace `<user>` with your username)

Even cooler (hard-to-believe, I know), you can view the sample image data with another http request
`http://sonic:8080/render-ws/v1/owner/<user>/project/iCAT_demo/stack/big_EM/z/0/box/0,0,2200,2200,0.5/jpeg-image`
where you specify a bounding box over the stack. You can replace the stack and the width, height, and zoom of the bounding box to render other views of stacks.

#### For a full, interactive list of the `render-ws` api, go to http://sonic:8080/swagger-ui/